In [41]:
import pandas as pd

# Read csv file
df = pd.read_csv("news_sample.csv")

# display head
#print(df.head())


In [42]:
import re

# a)
# six-digit date (DDMMYY) followed by 4 digit identifier (IIII)
# show either DDMMYYIIII or DDMMYY-IIII format


cpr_pattern = r'\b((?:0[1-9]|[12][0-9]|3[01]))((?:0[1-9]|1[0-2]))(\d{2})-?(\d{4})\b'

def extract_cpr(cpr_num):
    match = re.fullmatch(cpr_pattern, cpr_num)
    if match:
        return match.groups()
    else:
        return None

test_cprs = ["0102031234", "010203-1234", "3112995678", "311299-5678", "1234567890"]

for cpr in test_cprs:
    result = extract_cpr(cpr)
    if result:
        print(f"Valid CPR: {cpr} -> DD: {result[0]}, MM: {result[1]}, YY: {result[2]}, IIII: {result[3]}")
    else:
        print(f"Invalid CPR: {cpr}")

Valid CPR: 0102031234 -> DD: 01, MM: 02, YY: 03, IIII: 1234
Valid CPR: 010203-1234 -> DD: 01, MM: 02, YY: 03, IIII: 1234
Valid CPR: 3112995678 -> DD: 31, MM: 12, YY: 99, IIII: 5678
Valid CPR: 311299-5678 -> DD: 31, MM: 12, YY: 99, IIII: 5678
Invalid CPR: 1234567890
